# [Уменьшение количества цветов изображения](https://www.coursera.org/learn/vvedenie-mashinnoe-obuchenie/programming/yWVid/umien-shieniie-kolichiestva-tsvietov-izobrazhieniia)

## Введение

Самый распространенный тип задач машинного обучения — это задачи обучения с учителем. В них имеется обучающая выборка, для каждого объекта которой есть ответ, и нужно научиться предсказывать эти ответы для новых объектов. В такой постановке можно строго определить критерии качества.

Если же имеются лишь объекты, а ответов для них нет, то все равно можно пытаться найти в данных некую структуру. Задачи, которые ищут закономерности в неразмеченных выборках, называют задачами обучения без учителя. Типичный пример такой задачи — кластеризация, где требуется найти группы похожих объектов.

Кластеризация может использоваться для самых разных целей. В этом задании мы попробуем группировать схожие пиксели на изображении. Такой подход позволяет переходить к суперпиксельному представлению изображений, которое является более компактным и лучше подходит для решения ряда задач компьютерного зрения.

In [43]:
import pandas as pd
import numpy as np
from skimage.io import imread
import skimage
import pylab
from itertools import chain
from sklearn.cluster import KMeans
import math

%matplotlib inline

## 1. Загрузите картинку parrots.jpg. 
Преобразуйте изображение, приведя все значения в интервал от 0 до 1. Для этого можно воспользоваться функцией img_as_float из модуля skimage. Обратите внимание на этот шаг, так как при работе с исходным изображением вы получите некорректный результат.

In [32]:
image = imread('./data/parrots.jpg')
scale_image = skimage.img_as_float64(image)

width, height = len(scale_image[1]), len(scale_image)

## 2. Создайте матрицу объекты-признаки: характеризуйте каждый пиксель тремя координатами - значениями интенсивности в пространстве RGB.

In [33]:
listmerge = lambda lst: list(chain(*chain(*lst)))
val = listmerge(scale_image)
data = pd.DataFrame({'R': val[::3], 'G': val[1::3], 'B': val[2::3]})
data.head()

,R,G,B
0,0.015686,0.494118,0.019608
1,0.007843,0.494118,0.007843
2,0.007843,0.494118,0.007843
3,0.007843,0.494118,0.007843
4,0.007843,0.501961,0.011765


## 3. Запустите алгоритм K-Means с параметрами init='k-means++' и random_state=241. 
После выделения кластеров все пиксели, отнесенные в один кластер, попробуйте заполнить двумя способами: медианным и средним цветом по кластеру.

In [34]:
kmeans = KMeans(init='k-means++', random_state=241, n_jobs=-1)
kmeans8 = kmeans.fit(data)

/home/worker/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [35]:
cluster = pd.Series(kmeans8.labels_, name='lable')

In [36]:
data_c = data.join(cluster)
data_c.head()

,R,G,B,lable
0,0.015686,0.494118,0.019608,3
1,0.007843,0.494118,0.007843,3
2,0.007843,0.494118,0.007843,3
3,0.007843,0.494118,0.007843,3
4,0.007843,0.501961,0.011765,3


In [37]:
data_mean = data_c.copy()
for i in range(8):
    data_mean.loc[data_mean['lable'] == i, 'R'] = data_c[data_c['lable'] == i]['R'].mean()
    data_mean.loc[data_mean['lable'] == i, 'G'] = data_c[data_c['lable'] == i]['G'].mean()
    data_mean.loc[data_mean['lable'] == i, 'B'] = data_c[data_c['lable'] == i]['B'].mean()

data_mean.head()

,R,G,B,lable
0,0.095543,0.63545,0.066782,3
1,0.095543,0.63545,0.066782,3
2,0.095543,0.63545,0.066782,3
3,0.095543,0.63545,0.066782,3
4,0.095543,0.63545,0.066782,3


In [38]:
data_median = data_c.copy()
for i in range(8):
    data_median.loc[data_median['lable'] == i, 'R'] = data_c[data_c['lable'] == i]['R'].median()
    data_median.loc[data_median['lable'] == i, 'G'] = data_c[data_c['lable'] == i]['G'].median()
    data_median.loc[data_median['lable'] == i, 'B'] = data_c[data_c['lable'] == i]['B'].median()

data_median.head()

,R,G,B,lable
0,0.054902,0.643137,0.023529,3
1,0.054902,0.643137,0.023529,3
2,0.054902,0.643137,0.023529,3
3,0.054902,0.643137,0.023529,3
4,0.054902,0.643137,0.023529,3


## 4. Измерьте качество получившейся сегментации с помощью метрики PSNR. 
Эту метрику нужно реализовать самостоятельно ([см. определение](https://ru.wikipedia.org/wiki/%D0%9F%D0%B8%D0%BA%D0%BE%D0%B2%D0%BE%D0%B5_%D0%BE%D1%82%D0%BD%D0%BE%D1%88%D0%B5%D0%BD%D0%B8%D0%B5_%D1%81%D0%B8%D0%B3%D0%BD%D0%B0%D0%BB%D0%B0_%D0%BA_%D1%88%D1%83%D0%BC%D1%83)).

In [47]:
# MSE = lambda I, K, width, height: 1 / (3 * width * height) * sum([(i['R'] - j['R'])**2 + (i['G'] - j['G'])**2 + (i['B'] - j['B'])**2 for (_, i), (_, j)  in zip(I.iterrows(), K.iterrows())])
# mse = MSE(data_c, data_mean, width, height)
# PSNR = lambda MAX, MSE: 20 * math.log(MAX / math.sqrt(MSE))
# psnr_ = PSNR(1, mse)
# psnr_

42.452351239513895

In [49]:
MSE = lambda I, K, width, height: 1 / (3 * width * height) * sum([(i['R'] - j['R'])**2 + (i['G'] - j['G'])**2 + (i['B'] - j['B'])**2 for (_, i), (_, j)  in zip(I.iterrows(), K.iterrows())])
PSNR = lambda I, K, width, height, MAX: 20 * math.log(MAX / math.sqrt(MSE(I, K, width, height)))
# 42.452351239513895

In [50]:
psnr_mean = PSNR(data_c, data_mean, width, height, 1)
psnr_mean

42.452351239513895

In [52]:
psnr_median = PSNR(data_c, data_median, width, height, 1)
psnr_median

41.7687193038884

## 5. Найдите минимальное количество кластеров, при котором значение PSNR выше 20 (можно рассмотреть не более 20 кластеров, но не забудьте рассмотреть оба способа заполнения пикселей одного кластера). 
Это число и будет ответом в данной задаче.

In [56]:
means = []
medians = []
clustes = []
for count_cluster in range(8, 0, -1):
    kmeans = KMeans(n_clusters=count_cluster, init='k-means++', random_state=241, n_jobs=-1).fit(data)
    cluster = pd.Series(kmeans.labels_, name='lable')
    data_c = data.join(cluster)
    
    data_mean = data_c.copy()
    for i in range(8):
        data_mean.loc[data_mean['lable'] == i, 'R'] = data_c[data_c['lable'] == i]['R'].mean()
        data_mean.loc[data_mean['lable'] == i, 'G'] = data_c[data_c['lable'] == i]['G'].mean()
        data_mean.loc[data_mean['lable'] == i, 'B'] = data_c[data_c['lable'] == i]['B'].mean()
    
    psnr_mean = PSNR(data_c, data_mean, width, height, 1)

    data_median = data_c.copy()
    for i in range(8):
        data_median.loc[data_median['lable'] == i, 'R'] = data_c[data_c['lable'] == i]['R'].median()
        data_median.loc[data_median['lable'] == i, 'G'] = data_c[data_c['lable'] == i]['G'].median()
        data_median.loc[data_median['lable'] == i, 'B'] = data_c[data_c['lable'] == i]['B'].median()
    
    psnr_median = PSNR(data_c, data_median, width, height, 1)
    
    clustes.append(count_cluster)
    means.append(psnr_mean)
    medians.append(psnr_median)
    if psnr_mean < 20 and psnr_median < 20:
        break

In [58]:
means

[42.452351239513895,
 40.61988949794443,
 38.09394984674896,
 35.752674891165334,
 33.08314614583352,
 30.287066537555653,
 27.815617429588805,
 22.60870435595369]

In [59]:
medians

[41.7687193038884,
 39.91885694791713,
 36.959235459362084,
 34.942301883771385,
 32.24747319342532,
 29.442655086551394,
 26.83097499794993,
 21.70840490348916]